In [1]:
from abc import ABC
from pathlib import Path

import pandas as pd
from codetf.models import load_model_pipeline
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score
from sacrebleu import corpus_bleu, corpus_chrf, corpus_ter
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline

/home/paul/.conda/envs/codetf/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
root_dir = Path.cwd()

# Create Predictions
Im folgenden wird mit allen Models für den Testdatensatz die Predictions generiert.

In [3]:
class AbstractModel(ABC):
    def predict(self, code: str) -> str:
        """Predicts the given code ."""
        raise NotImplementedError()
    
    def model_name(self) -> str:
        """Returns the model name ."""
        raise NotImplementedError()
    
class CodeTFModel(AbstractModel):
    def __init__(self, model_name: str, model_type: str, task: str) -> None:
        """Initialize the pipeline ."""
        super().__init__()

        self._model = load_model_pipeline(model_name=model_name, model_type=model_type, task=task)
        self._model_name = model_name
        self._model_type = model_type
        self._task = task

    def predict(self, code: str) -> str:
        """Predicts the given code ."""
        return self._model.predict([code])[0]
    
    def model_name(self) -> str:
        """The name of the model ."""
        return f"{self._model_name}-{self._model_type}-{self._task}"
    
class SebisModel(AbstractModel):
    def __init__(self, model_name: str) -> None:
        """Initialize the pipeline ."""
        super().__init__()

        self._pipeline = SummarizationPipeline(
            model=AutoModelWithLMHead.from_pretrained(model_name),
            tokenizer=AutoTokenizer.from_pretrained(model_name, skip_special_tokens=True),
            device=0
        )
        self._model_name = model_name

    def predict(self, code: str) -> str:
        """Predict the text for a given code ."""
        return self._pipeline([code])[0]["summary_text"]
    
    def model_name(self) -> str:
        """The model name of the model ."""
        return self._model_name.replace("/", "-")

In [4]:
def get_preds(df: pd.DataFrame, model: AbstractModel):
    """Get predictions from a model ."""
    file_path = root_dir / "data" / "preds" / f"{model.model_name()}.csv"
    
    if file_path.exists():
        return

    df = df.copy()
    df["pred"] = df["code"].map(model.predict)
    df[["ref", "pred"]].to_csv(file_path)

In [5]:
dataset = load_dataset("json", data_files={
    "test": str(root_dir.parent / "data" / "test.jsonl"),
}, cache_dir=root_dir.parent / "data" / "cache")

Found cached dataset json (/home/paul/projects/edu/master/mdl-ii/src/data/cache/json/default-acdd91729f392843/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
100%|██████████| 1/1 [00:00<00:00, 17.64it/s]


In [6]:
def inference(doc):
    """Add the ref to the doc ."""
    doc["ref"] = [" ".join(docstring) for docstring in doc["docstring_tokens"]]
    return doc    

dataset = dataset.map(inference, batched=True)
dataset.set_format(type="pandas", columns=["ref", "code"])
df = dataset["test"][:]
df.head()

Loading cached processed dataset at /home/paul/projects/edu/master/mdl-ii/src/data/cache/json/default-acdd91729f392843/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-e06db1d51f6ed421.arrow


,code,ref
0,def sina_xml_to_url_list(xml_data):\n rawur...,str - > list Convert XML to URL List . From Bi...
1,"def dailymotion_download(url, output_dir='.', ...",Downloads Dailymotion videos by URL .
2,"def sina_download(url, output_dir='.', merge=T...",Downloads Sina videos by URL .
3,"def sprint(text, *colors):\n return ""\33[{}...",Format text with color or other effects into A...
4,"def print_log(text, *colors):\n sys.stderr....",Print a log message to standard error .


## CodeT5 Base

In [7]:
get_preds(df, CodeTFModel(model_name="codet5", model_type="base-multi-sum", task="pretrained"))

In [8]:
get_preds(df, CodeTFModel(model_name="codet5", model_type="base", task="sum_python"))

## T5 Small

In [11]:
get_preds(df, SebisModel("SEBIS/code_trans_t5_small_code_documentation_generation_python"))

In [12]:
get_preds(df, SebisModel("SEBIS/code_trans_t5_small_code_documentation_generation_python_transfer_learning_finetune"))

In [13]:
get_preds(df, SebisModel("SEBIS/code_trans_t5_small_code_documentation_generation_python_multitask"))

In [14]:
get_preds(df, SebisModel("SEBIS/code_trans_t5_small_code_documentation_generation_python_multitask_finetune"))

## T5 Base

In [15]:
get_preds(df, SebisModel("SEBIS/code_trans_t5_base_code_documentation_generation_python"))

In [16]:
get_preds(df, SebisModel("SEBIS/code_trans_t5_base_code_documentation_generation_python_transfer_learning_finetune"))

In [17]:
get_preds(df, SebisModel("SEBIS/code_trans_t5_base_code_documentation_generation_python_multitask"))

In [18]:
get_preds(df, SebisModel("SEBIS/code_trans_t5_base_code_documentation_generation_python_multitask_finetune"))

## T5 Large

In [7]:
get_preds(df, SebisModel("SEBIS/code_trans_t5_large_code_documentation_generation_python_transfer_learning_finetune"))

/home/paul/.conda/envs/codetf/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:1363: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [8]:
get_preds(df, SebisModel("SEBIS/code_trans_t5_large_code_documentation_generation_python_multitask"))

In [9]:
get_preds(df, SebisModel("SEBIS/code_trans_t5_large_code_documentation_generation_python_multitask_finetune"))

# Auswertung

In [3]:
def get_scores(df: pd.DataFrame, model_name: str):
    """Get scores for a given model ."""
    scorer = rouge_scorer.RougeScorer(["rouge1", "rougeL"], use_stemmer=True)
    scores = [scorer.score(ref, hyp) for ref, hyp in zip(df["ref"].to_list(), df["pred"].to_list())]
    rouge1 = sum([score["rouge1"].fmeasure for score in scores]) / len(scores)
    rougeL = sum([score["rougeL"].fmeasure for score in scores]) / len(scores)
    # meteor = sum([meteor_score([ref], hyp) for ref, hyp in zip(df["ref"].to_list(), df["pred"].to_list())]) / len(df["pred"].to_list())

    return pd.DataFrame({
        "bleu": corpus_bleu(df["pred"].to_list(), [df["ref"].to_list()]).score,
        "chrf": corpus_chrf(df["pred"].to_list(), [df["ref"].to_list()]).score,
        "ter": corpus_ter(df["pred"].to_list(), [df["ref"].to_list()]).score,
        "rouge1": rouge1, "rougeL": rougeL
    }, index=pd.Index([model_name], name="Model"))

In [ ]:
df = pd.DataFrame(columns=["bleu", "chrf", "ter"])

for csv_file in sorted((root_dir / "data" / "preds").glob("*.csv"), key=lambda f: f.name):
    df = pd.concat([df, get_scores(pd.read_csv(csv_file, index_col=0), csv_file.name)])

In [5]:
pd.options.display.float_format = '{:.3f}'.format
df

,bleu,chrf,ter,rouge1,rougeL
SEBIS-code_trans_t5_base_code_documentation_generation_python.csv,4.638,23.194,102.021,0.263,0.234
SEBIS-code_trans_t5_base_code_documentation_generation_python_multitask.csv,2.957,15.566,93.160,0.221,0.208
SEBIS-code_trans_t5_base_code_documentation_generation_python_multitask_finetune.csv,13.766,33.452,78.742,0.443,0.410
SEBIS-code_trans_t5_base_code_documentation_generation_python_transfer_learning_finetune.csv,21.671,37.954,71.560,0.485,0.457
SEBIS-code_trans_t5_large_code_documentation_generation_python_multitask.csv,13.487,32.527,79.615,0.433,0.401
SEBIS-code_trans_t5_large_code_documentation_generation_python_multitask_finetune.csv,16.362,35.033,80.671,0.445,0.412
SEBIS-code_trans_t5_large_code_documentation_generation_python_transfer_learning_finetune.csv,23.306,38.984,69.745,0.497,0.470
SEBIS-code_trans_t5_small_code_documentation_generation_python.csv,5.495,25.007,103.981,0.287,0.256
SEBIS-code_trans_t5_small_code_documentation_generation_python_multitask.csv,5.450,20.280,89.910,0.295,0.276
SEBIS-code_trans_t5_small_code_documentation_generation_python_multitask_finetune.csv,16.378,34.692,76.738,0.452,0.421
